In [1]:
import warp as wp
wp.init()
import numpy as np

Warp 0.6.1 initialized:
   CUDA Toolkit: 11.5, Driver: 11.7
   Devices:
     "cpu"    | Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
     "cuda:0" | Quadro P1000 (sm_61)
   Kernel cache: C:\Users\20192037\AppData\Local\NVIDIA Corporation\warp\Cache\0.6.1


### Test code

In [87]:
train_img = np.array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          3,  18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  30,  36,
         94, 154, 170, 253, 253, 253, 253, 253, 225, 172, 253, 242, 195,
         64,   0,   0,   0,   0,   0,   0,   0,   0], dtype=np.int16)
test_img = np.array([ 93,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253, 253,
        253, 253, 218,  30,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0, 169, 253, 253, 253,
        213, 142, 176, 253, 253, 122,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  52, 250, 253,
        210,  32,  12,   0,   6, 206, 253, 140,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=np.int16)

In [88]:
@wp.kernel
def sq_add(train_img: wp.array(dtype=float),
           test_img: wp.array(dtype=float),
           sq_sum: wp.array(dtype=float)):

   # thread index
   tid = wp.tid()
    
   sq_abs = wp.pow(train_img[tid] - test_img[tid], 2.0)
   
   wp.atomic_add(sq_sum, 0, sq_abs)

In [95]:
print(sum(i**2 for i in train_img-test_img))

wptrain = wp.from_numpy(train_img, dtype=float)
wptest = wp.from_numpy(test_img, dtype=float)
wpresults = wp.array([0], dtype=float)

wp.launch(kernel=sq_add,
          dim=len(train_img),
          inputs=[wptrain, wptest, wpresults])

print(wpresults.numpy())

1583682
[1583682.]


In [2]:
@wp.kernel
def img_norm(train_imgs: wp.array(dtype=float, ndim=2),
           test_img: wp.array(dtype=float),
           n: int,
           norms: wp.array(dtype=float)):

    # thread index
    tid = wp.tid()

    train_img = train_imgs[tid]

    s = float(0)
    for i in range(n):
        diff = train_img[i] - test_img[i]
        s += wp.pow(diff, 2.0)
    
    norms[tid] = wp.sqrt(s)

In [99]:
print(np.linalg.norm(train_img-test_img))

train_imgs = np.tile(train_img, (10,1))
wptrain = wp.from_numpy(train_imgs, dtype=float)
wptest = wp.from_numpy(test_img, dtype=float)
wpresults = wp.empty(shape=len(train_imgs), dtype=float)

wp.launch(kernel=img_norm,
          dim=len(train_imgs),
          inputs=[wptrain, wptest, len(test_img), wpresults])

print(wpresults.numpy())

1258.4442776698538
[1258.4443 1258.4443 1258.4443 1258.4443 1258.4443 1258.4443 1258.4443
 1258.4443 1258.4443 1258.4443]


### Run code

In [3]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
from collections import Counter
from time import perf_counter

In [26]:
def get_images(name, size):
    images = np.empty((size, 784), dtype=np.int16)
    numbers = np.empty(size, dtype=np.int32)

    with open(name, 'rt') as file:
        for i in range(size):
            row = file.readline().split(',')

            numbers[i] = row[0]
            images[i] = row[1:]

    return numbers, images

def mink_dist(x, y, p=2):
    return np.linalg.norm(x - y, ord=p, axis=-1)

not_eq = lambda x,y: x != y

def k_nearest_neighbours(train_images, train_numbers, test_image, k, dist_func):
    norm_list = dist_func(train_images, test_image)
    nearest_idx = np.empty(k, dtype=np.int32)
    common_nr = np.empty(k, dtype=np.int8)
    for i in range(k):
        min_idx = np.argmin(norm_list)
        norm_list[min_idx] = np.inf
        nearest_idx[i] = min_idx

        common_nr[i] = Counter(train_numbers[nearest_idx[:i+1]]).most_common(1)[0][0]

    return common_nr

def k_nearest_neighbours_gpu(wpimages_train, train_numbers, wpimg, k):
    wpimages_train_len = wpimages_train.shape[0]
    wpresults = wp.empty(shape=wpimages_train_len, dtype=float, device='cuda:0')

    wp.launch(kernel=img_norm,
          dim=wpimages_train_len,
          inputs=[wpimages_train, wpimg, wpimg.size, wpresults],
          device='cuda:0')

    norm_list = wpresults.numpy()

    nearest_idx = np.empty(k, dtype=np.int32)
    common_nr = np.empty(k, dtype=np.int8)
    for i in range(k):
        min_idx = np.argmin(norm_list)
        norm_list[min_idx] = np.inf
        nearest_idx[i] = min_idx

        common_nr[i] = Counter(train_numbers[nearest_idx[:i+1]]).most_common(1)[0][0]

    return common_nr

def k_nearest_neighbours_LOOCV(train_images, train_numbers, test_image, k, dist_func, index):
    """ 
    Applies the k-NN approach, while disregarding index from the train images (LOOCV)
    """
    assert 0 <= index and index < train_images.size, "Index out of range"
    
    return k_nearest_neighbours(np.delete(train_images, index, axis = 0), np.delete(train_numbers, index), test_image, k, dist_func)

def cross_validation(train_images, train_numbers, k_range, dist_func):
    """
    Computes the cross-validation score of the training dataset for each k in the range 1 to k_range
    """

    # Prediction rule for image at index while leaving index out of training
    neighbour_LOOCV_func = lambda index: k_nearest_neighbours_LOOCV(train_images, train_numbers, train_images[index], k_range, dist_func, index)

    train_common_nrs_LOOCV = np.array(list(map(neighbour_LOOCV_func, range(train_numbers.size)))).transpose()

    LOOCV_scores = np.sum(not_eq(train_common_nrs_LOOCV, train_numbers), axis=1)/train_numbers.size

    return LOOCV_scores

In [5]:
numbers_train, images_train = get_images('data/MNIST_train.csv', 60000)
numbers_test, images_test = get_images('data/MNIST_test.csv', 10000)

0.6284270999999961


In [7]:
img = images_test[0]
wpimages_train = wp.from_numpy(images_train, dtype=float, device='cuda:0')
wpimg = wp.from_numpy(img, dtype=float, device='cuda:0')
wpresults = wp.empty(shape=len(images_train), dtype=float, device='cuda:0')

In [33]:
%timeit k_nearest_neighbours(images_train, numbers_train, img, 20, lambda x,y: mink_dist(x,y,2))

463 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit k_nearest_neighbours_gpu(wpimages_train, numbers_train, wpimg, 20)

45.3 ms ± 220 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
